Here we have investigate the behaviour of the non-linear problem:

\begin{equation}
\begin{cases}
     \nu\boldsymbol{u} + \beta \lvert \boldsymbol{u}\rvert \boldsymbol{u} + \nabla p = \boldsymbol{f} \;\;\; \text{in} \;\; \Omega ,\\ \text{div} \,  \boldsymbol{u} = 0\,, \;\; \text{in} \;\;\; \Omega \\
     p = p_D \;\; \text{on} \;\; \partial\Omega
\end{cases}
\end{equation}

We defined the functional spaces:

\begin{equation*}
     \boldsymbol{X} = \{ \boldsymbol{u} \in [L^3(\Omega)]^d \;\;\text{s.t.} \;\;\text{div} \ \boldsymbol{u} \in L^2(\Omega) \,\}, \quad \|\boldsymbol{u}\|_X = \|\boldsymbol{u}\|_{L^3} + \|\text{div}\boldsymbol{u}\|_{L^2} \\
     M = L^2(\Omega), \quad \|p\|_M = \|p\|_{L^2}  
\end{equation*}

and we derived the weak formulation:

\begin{equation*}
    \text{Find}\;\; (\boldsymbol{u},p) \in \boldsymbol{X} \times M\;\; \text{s.t.}\;\;
\end{equation*}
     \begin{equation}
     \begin{cases}
    \int_\Omega (\nu\boldsymbol{u} + \beta|\boldsymbol{u}|\boldsymbol{u}) \cdot \boldsymbol{v} - \int_\Omega p\,\text{div}\boldsymbol{v} = \int_\Omega \boldsymbol{f} \cdot \boldsymbol{v} -\int_{\partial\Omega} p_D \boldsymbol{v} \cdot \boldsymbol{n} \quad\quad \forall\boldsymbol{v} \in \boldsymbol{X} \\
    \int_\Omega q\,\text{div}\boldsymbol{u} = 0 \quad\quad \forall q \in M
\end{cases}
\end{equation}

and the associated discrete formulation:

\begin{equation*}
    \text{Find}\;\; (\boldsymbol{u}_h,p_h) \in \boldsymbol{X}_h \times M_h\;\; \text{s.t.}\;\;
\end{equation*}
\begin{equation}
     \begin{cases}
    \int_\Omega (\nu\boldsymbol{u}_h + \beta|\boldsymbol{u}_h|\boldsymbol{u}_h) \cdot \boldsymbol{v}_h - \int_\Omega p_h\text{div}\boldsymbol{v}_h = \int_\Omega \boldsymbol{f} \cdot \boldsymbol{v}_h -\int_{\partial\Omega} p_D \boldsymbol{v}_h \cdot \boldsymbol{n} \quad\quad \forall\boldsymbol{v}_h \in \boldsymbol{X}_h \\
    \int_\Omega q_h\text{div}\boldsymbol{u}_h = 0 \quad\quad \forall q_h \in M_h
    \end{cases}
\end{equation}

where the discrete spaces are taken as:

\begin{equation*}
    \boldsymbol{X}_h = \{\boldsymbol{v}_h\in \boldsymbol{H}(\text{div};\mathcal{T}_h)\,, \;\;\boldsymbol{v}_h|_{\mathcal{K}} \in \mathbb{R}\mathbb{T}^{\,r}(\mathcal{K})\,,\;\;\forall\mathcal{K}\in\mathcal{T}_h\}\,,
\end{equation*}
\begin{equation*}
    M_h = \mathbb{P}^{\,r-1}({\mathcal{T}_h})\,
\end{equation*}

In [ ]:
%%capture
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

In [ ]:
from fenics import *
from mshr import *
import matplotlib.pyplot as plt
import numpy as np
import math

In [ ]:
# Data of the experiment
nu = Constant(10.0)
beta = Constant(5)

f = Expression(('-beta*(1/nu)*(1/nu)*sqrt(pow(x[0],2)+pow(x[1],2))*x[1]',
                      '-beta*(1/nu)*(1/nu)*sqrt(pow(x[0],2)+pow(x[1],2))*x[0]'), degree=4, nu=nu, beta=beta) #Forcing term
p_exact = Expression('x[0]*x[1]', degree=2)
u_exact = Expression(('-(1/nu)*x[1]', '-(1/nu)*x[0]'), degree=3, nu=nu)

d= 100
degree = 1

In [ ]:
def solve_ForchHeatCoupled(d, degree, nu, beta, u_exact, p_exact, f):

    # 1. mesh generation and definition of the error vector
    ERRORI = np.empty([1,1000])
    square = Rectangle(Point(0.0, 0.0), Point(1.0, 1.0))
    mesh = generate_mesh(square, d)
    n = FacetNormal(mesh)

    # 2. definition of a MFE stable coupling
    V = FiniteElement('RT', mesh.ufl_cell(), degree)
    Q = FiniteElement('DG', mesh.ufl_cell(), degree-1)
    W = FunctionSpace(mesh, MixedElement([V, Q]))

    # 3. define formulation for initial guess (linear Darcy pb)
    u, p = TrialFunctions(W)
    v, q = TestFunctions(W)
    w_old = Function(W)

    a_in = ((nu) * dot(u, v) - q * div(u) - p * div(v)) * dx
    L = dot(f, v) * dx - p_exact * dot(v, n) * ds
    solve(a_in == L, w_old)
    u_old, p_old = w_old.split()

    # 4. define meta-parameters and solve problem by fixed-point linearization
    niter = 1000
    tolerance = 1e-8
    j = 0

    for i in range(niter):
        u, p = TrialFunctions(W)
        w = Function(W)
        a = ((nu + beta*sqrt(inner(u_old,u_old))) * dot(u, v) - q * div(u) - p * div(v)) * dx
        solve(a == L, w)
        u, p = w.split()
        error = (errornorm(u, u_old, 'Hdiv') / norm(u, 'Hdiv') +
                 errornorm(p, p_old, 'L2') / norm(p, 'L2'))
        print(' step={}: {:.3e}'.format(i, error))
        ERRORI[0,j] = error
        j = j+1
        w_old = w
        u_old, p_old = w_old.split()

        if error < tolerance:
            break

    # 5. compute error
    l2err_p = errornorm(p_exact, p_old, 'L2')
    l2err_u = errornorm(u_exact, u_old, 'L2')
    Hdiverr_u = errornorm(u_exact, u_old, 'Hdiv0')
    u = u_old

    return u, p_old, l2err_p, l2err_u, Hdiverr_u, ERRORI, j

We plot the logarithm of the error versus the number of iterations for the convergence

In [ ]:
iter = np.empty([1,step])
for a in range(step):
 iter[0,a]=a

In [ ]:
err = np.empty([1,step])
for c in range(step):
 err[0,c] = ERROR[0,c]

In [ ]:
plt.semilogy(iter,err,'bo')

plt.title("Error Vs Number of iterations");
plt.xlabel("Iterations")
plt.ylabel("Error")

In this last part we have gathered all the results that we have obtained and we plotted $\beta$ versus the number of iterations

In [ ]:
beta_vect = np.array([0.01, 0.1, 1, 10, 20, 30, 40, 50, 60, 80, 100])
iter_vect = np.array([3, 5, 11, 59, 108, 156, 203, 250, 297,391, 484])

In [ ]:
plt.plot(beta_vect,iter_vect,'ro')

plt.title("Beta VS Number of iterations");
plt.xlabel("Beta")
plt.ylabel("Iterations")